In [1]:
# Libraries

import os
import cv2
import numpy as np
import pandas as pd
import nibabel as nib
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Dropout, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50

from sklearn.model_selection import train_test_split

import dotenv

2024-11-09 16:02:42.983629: E external/local_xla/xla/stream_executor/plugin_registry.cc:93] Invalid plugin kind specified: DNN


### Hyperparameters

In [2]:
# Load environment variables
dotenv.load_dotenv()

EPOCHS = 50
BATCH_SIZE = 512

### Import Dataset

In [3]:
training_dataset_path = os.path.join(os.environ['DATASET_PATH'], 'Training')

class_labels = {'glioma': 0, 'meningioma': 1, 'pituitary': 2, 'notumor': 3}
image_folders = ['glioma', 'meningioma', 'notumor', 'pituitary']
class_labels_reverse = {v: k for k, v in class_labels.items()}


In [4]:
# Initialize lists to store images and labels for the training dataset
images = []
labels = []

# Loop over the folders in the Training dataset to load images and their labels
for folder in image_folders:
    folder_path = os.path.join(training_dataset_path, folder)
    
    # List image files in the folder
    image_files = sorted(os.listdir(folder_path))

    for img_file in image_files:
        img_path = os.path.join(folder_path, img_file)

        # Read the image (grayscale because MRI data)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

        # Resize image if necessary (standardize image size)
        img = cv2.resize(img, (128, 128))  # Resize to 128x128 or any size you prefer

        # Append image and label (use list append method)
        images.append(img)
        labels.append(class_labels[folder])  # Assign label based on folder name

# Convert lists to numpy arrays after appending all images and labels
images = np.array(images)
labels = np.array(labels)

# Normalize the images (optional but helps with training)
images = images.astype('float32') / 255.0

# Convert images to 3 channels (grayscale to RGB) for ResNet50
X_rgb = np.repeat(images.reshape(-1, 128, 128, 1), 3, axis=-1)

# Split the dataset into training and validation sets (80% for training, 20% for validation)
X_train, X_val, y_train, y_val = train_test_split(X_rgb, labels, test_size=0.2, random_state=42)

# Load pre-trained ResNet50 model (without the top layers)
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

# Freeze the base model layers
base_model.trainable = False

In [5]:
# Add custom classifier layers on top of the base model
x = base_model.output
x = Flatten()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(4, activation='softmax')(x)  # 4 classes

# Create the full model
model = Model(inputs=base_model.input, outputs=x)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Print model summary
model.summary()

Model: "model"


__________________________________________________________________________________________________


 Layer (type)                Output Shape                 Param #   Connected to                  


 input_1 (InputLayer)        [(None, 128, 128, 3)]        0         []                            


 conv1_pad (ZeroPadding2D)   (None, 134, 134, 3)          0         ['input_1[0][0]']             


...

 dense_1 (Dense)             (None, 4)                    2052      ['dropout[0][0]']             


Total params: 40367492 (153.99 MB)


Trainable params: 16779780 (64.01 MB)


Non-trainable params: 23587712 (89.98 MB)


__________________________________________________________________________________________________


In [6]:
# Train the model using the training data
history = model.fit(X_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_data=(X_val, y_val))

# Evaluate the model on the validation (split) set
val_loss, val_acc = model.evaluate(X_val, y_val)
print(f"Validation Loss: {val_loss}")
print(f"Validation Accuracy: {val_acc}")


Epoch 1/50


I0000 00:00:1731186191.082074   13142 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1/9 [==>...........................] - ETA: 2:16 - loss: 1.5804 - accuracy: 0.2461

2/9 [=====>........................] - ETA: 2s - loss: 1.5656 - accuracy: 0.2588  

3/9 [=========>....................] - ETA: 2s - loss: 1.5287 - accuracy: 0.2747

...

25/36 [===================>..........] - ETA: 0s - loss: 0.4460 - accuracy: 0.8213

29/36 [=======================>......] - ETA: 0s - loss: 0.4467 - accuracy: 0.8190

33/36 [==========================>...] - ETA: 0s - loss: 0.4526 - accuracy: 0.8229

36/36 [==============================] - 2s 42ms/step - loss: 0.4507 - accuracy: 0.8233


Validation Loss: 0.4507012665271759
Validation Accuracy: 0.8232721090316772


In [7]:
def test_image(model, image_path):
    # Load and preprocess the input image
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (128, 128))  # Resize to match training input size
    img = img.astype('float32') / 255.0  # Normalize
    img = np.repeat(img, 3, axis=-1)  # Convert grayscale to RGB
    img = img.reshape(1, 128, 128, 3)  # Add batch dimension
    
    # Predict the class
    prediction = model.predict(img)
    predicted_class = np.argmax(prediction, axis=1)  # Get the class index with the highest probability

    # Return the predicted label
    predicted_label = class_labels_reverse[predicted_class[0]]  # Map the class index to label
    return predicted_label

test_image_path1 = os.path.join(os.environ['DATASET_PATH'], 'Testing/glioma/Te-gl_0010.jpg')
test_image_path2 = os.path.join(os.environ['DATASET_PATH'], 'Testing/meningioma/Te-me_0018.jpg')
test_image_path3 = os.path.join(os.environ['DATASET_PATH'], 'Testing/notumor/Te-no_0027.jpg')
test_image_path4 = os.path.join(os.environ['DATASET_PATH'], 'Testing/pituitary/Te-pi_0094.jpg')

predicted_label1 = test_image(model, test_image_path1)
predicted_label2 = test_image(model, test_image_path2)
predicted_label3 = test_image(model, test_image_path3)
predicted_label4 = test_image(model, test_image_path4)

print(f"Predicted label: {predicted_label1}")
print(f"Predicted label: {predicted_label2}")
print(f"Predicted label: {predicted_label3}")
print(f"Predicted label: {predicted_label4}")



1/1 [==============================] - ETA: 0s

1/1 [==============================] - 0s 487ms/step


1/1 [==============================] - ETA: 0s

1/1 [==============================] - 0s 17ms/step


1/1 [==============================] - ETA: 0s

1/1 [==============================] - 0s 15ms/step


1/1 [==============================] - ETA: 0s

1/1 [==============================] - 0s 14ms/step


Predicted label: glioma
Predicted label: notumor
Predicted label: notumor
Predicted label: pituitary
